In [1]:
import networkx as nx
import sqlite3
import pathSearch, db

# 1. 領域指定
points = [
    [35.20902036505841, 136.86844825744632],
    [35.140265534798395, 137.02620506286624]
]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)

# 2. 道路データ取得（link, length形式の取得）
link, length = db.getRectangleRoadData(y1, x1, y2, x2)

G = pathSearch.linkToGraph(link, length)
pathSearch.connectGraph(G)
sp = pathSearch.ShortestPathFinder(G)

chosen_indices, subset_list = pathSearch.set_cover(points, 0.3, sp)
for idx in chosen_indices:
    print(subset_list[idx])



{'[35.2068686, 136.8688981]', '[35.2075966, 136.8690422]', '[35.2091581, 136.8715567]', '[35.208352, 136.8712862]', '[35.207361, 136.869722]', '[35.207938, 136.871159]', '[35.2071975, 136.8682761]', '[35.2091687, 136.8700098]', '[35.2078445, 136.8697635]', '[35.2087484, 136.8714234]', '[35.2078057, 136.8684029]', '[35.2095485, 136.8687254]', '[35.2093444, 136.8693652]'}
{'[35.140371, 137.0248696]', '[35.1423538, 137.0246496]', '[35.1414107, 137.0238128]', '[35.1407527, 137.0247838]', '[35.1427674, 137.0264922]', '[35.1415949, 137.0261141]', '[35.1415949, 137.0241776]', '[35.1399673, 137.0257787]', '[35.1415773, 137.0246443]', '[35.1412045, 137.0261034]', '[35.1415817, 137.0247355]', '[35.140478, 137.024217]', '[35.1419678, 137.0239308]', '[35.1412045, 137.0247462]', '[35.1419721, 137.0261088]', '[35.1405743, 137.0243348]', '[35.139985, 137.0261195]', '[35.140376, 137.023819]', '[35.1419634, 137.0246496]', '[35.140255, 137.023228]', '[35.1407527, 137.026098]', '[35.1395551, 137.0257815]

In [16]:
import networkx as nx
import heapq
from collections import defaultdict

def optimal_vehicle_stops(G, user_queries, start, goal):
    users = [f"user{i}" for i in range(len(user_queries))]
    user_from = {u:q[0] for u, q in zip(users, user_queries)}
    user_to   = {u:q[1] for u, q in zip(users, user_queries)}
    
    # 最短距離テーブル全 pairs
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight="weight"))

    # DPテーブル
    dp = defaultdict(dict)
    pq = []

    # 初期状態 : 全員徒歩。乗車集合空っぽ
    init_state = frozenset()
    base_cost = 0
    for i,u in enumerate(users):
        base_cost += shortest_paths[user_from[u]][start]  # 徒歩: 乗車まで
        base_cost += shortest_paths[user_to[u]][goal]     # 徒歩: 降車後
    heapq.heappush(pq, (base_cost, start, init_state, None, None, [start]))
    dp[start][init_state] = (base_cost, None, None, [start])

    # 事前: 状態空間生成補助用
    from itertools import chain, combinations
    def powerset(iterable):
        "powerset([a,b,c]) --> (), (a,), (b,), (c,), (a,b), (a,c), ..."
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    
    # メインループ
    while pq:
        cost_here, node, state, parent_node, parent_state, stops = heapq.heappop(pq)
        # goal終点で全員降りたら完了
        if node == goal and len(state)==0:
            # 解復元
            path = []
            cur_node, cur_state = node, state
            cur_stops = stops
            return list(cur_stops), cost_here

        # dp最適値より悪い場合はskip
        if cost_here > dp[node][state][0]:
            continue
        
        for next_node in G.neighbors(node):
            edge_cost = G[node][next_node]["weight"]

            # 状態遷移: 乗っているユーザ情報に対し、誰を降ろし/誰を乗せるか全部チェック
            # 乗ってない人で「from == next_node」→乗せる/既に乗ってる人で「to==next_node」→降ろす/乗る&同時も可
            # 全ユーザごとに今board, 今boardで誰をおろす/誰をのせるかでパワーセット（各降車集合,各乗車集合）

            cur_onboard = set(state)
            # 乗車候補: not in cur_onboard かつ from==next_node
            geton_candidates = [u for u in users if (u not in cur_onboard and user_from[u]==next_node)]
            getoff_candidates= [u for u in cur_onboard if user_to[u]==next_node]

            # 乗る/降りる サブセット全列挙（理論上2^(降車数+乗車数)パターン。ただし同時に定義域を絞る）
            for geton_set in powerset(geton_candidates):
                for getoff_set in powerset(getoff_candidates):
                    # 状態更新
                    new_state = (cur_onboard | set(geton_set)) - set(getoff_set)
                    new_state = frozenset(new_state)

                    # 停車点: 誰か乗/降車する場合はstopsに含める, 何もしなければ道なり
                    new_stops = list(stops)
                    if geton_set or getoff_set:
                        if next_node not in new_stops:
                            new_stops = new_stops + [next_node]

                    # コスト計算
                    new_cost = cost_here + edge_cost

                    # 新たに乗る→徒歩分を減点, 降ろす→徒步分を減点
                    for u in geton_set:
                        new_cost -= shortest_paths[user_from[u]][start]
                    for u in getoff_set:
                        new_cost -= shortest_paths[user_to[u]][goal]
                    
                    # DP表更新
                    if new_state not in dp[next_node] or new_cost < dp[next_node][new_state][0]:
                        dp[next_node][new_state] = (new_cost, node, state, new_stops)
                        heapq.heappush(pq, (new_cost, next_node, new_state, node, state, new_stops))
    return None, float('inf')

# 使い方例（仮想: 任意個数クエリ可）
if __name__ == '__main__':
    G = nx.Graph()
    edges = [
        ("a", "b", 1),
        ("a", "c", 3),
        ("b", "c", 5),
        ("b", "d", 2),
        ("c", "d", 5),
        ("c", "e", 7),
        ("d", "e", 6),
        ("d", "f", 4),
        ("e", "f", 1),
        ("e", "g", 8),
        ("f", "g", 3),
        ("f", "h", 4),
        ("g", "h", 2)
    ]
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)

    # 例: 任意個数OK
    user_queries = [
        ("b", "f"),
        ("c", "g"),
        # ("d", "h"),
        # ("e", "b"),
    ]
    stops, total_cost = optimal_vehicle_stops(G, user_queries, "a", "h")
    print("最適停車点順:", stops)
    print("合計コスト:", total_cost)

最適停車点順: ['a', 'b', 'f']
合計コスト: 16


In [ ]:
#コピー 徒歩移動距離制約追加する前
import heapq
import itertools
"""
経路探索などを行う
"""
import sqlite3
import networkx as nx
import heapq
import itertools
import ast

class ShortestPathFinder:
    len_dic = {}
    kdtree = None
    kdtree_nodes = None
    kdtree_coords_km = None
    transformer = None
    zone = None

    def __init__(self, G, use_db_cache=False, db_path="paths.db", weight="weight", bulk_size=100, utm_zone=54):
        self.G = G
        self.weight = weight
        self.use_db_cache = use_db_cache
        self.bulk_size = bulk_size
        self._bulk_buffer = []
        self.zone = utm_zone

        # DBキャッシュ
        if self.use_db_cache:
            self.conn = sqlite3.connect(db_path)
            self._init_db()
        else:
            self.conn = None

        print("graph constructed")

    def _init_db(self):
        cur = self.conn.cursor()
        cur.execute('''
            CREATE TABLE IF NOT EXISTS paths (
                coord1 TEXT,
                coord2 TEXT,
                length REAL,
                PRIMARY KEY (coord1, coord2)
            )
        ''')
        cur.execute('CREATE INDEX IF NOT EXISTS idx_coord1 ON paths(coord1)')
        cur.execute('CREATE INDEX IF NOT EXISTS idx_pair ON paths(coord1, coord2)')
        self.conn.commit()

    def _latlon_to_km(self, latlon):
        # 入力: [lat, lon]リスト
        x, y = type(self).transformer.transform(latlon[1], latlon[0])
        return [x/1000, y/1000]

    def nearestNode(self, p):
        """
        p: [lat, lon]リスト。KDTreeで最近傍ノード（ノードのstr型＝"[lat, lon]"）を返す。
        """
        coord_km = self._latlon_to_km(p)
        dist, idx = type(self).kdtree.query(coord_km)
        return type(self).kdtree_nodes[idx]  # 返値はstr型

    def nodes_within_radius(self, p, r_km):
        """
        p: "[lat, lon]"形式str, r_km: 距離km
        返値: "[lat, lon]" 形式strのリスト
        """
        latlon = ast.literal_eval(p)  # "[lat,lon]" → [lat, lon]リスト
        coord_km = self._latlon_to_km(latlon)
        idxs = type(self).kdtree.query_ball_point(coord_km, r_km)
        return [type(self).kdtree_nodes[i] for i in idxs]
    
    def euclidean(self, a, b):
        a = list(map(float, a.strip("[]").split(',')))
        b = list(map(float, b.strip("[]").split(',')))
        return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5
    
    def SP(self, st, en):
        return nx.astar_path(self.G, st, en, heuristic=self.euclidean)

    def len_SP(self, node1, node2):
        # メモリキャッシュ
        try:
            return type(self).len_dic[node1][node2]
        except KeyError:
            pass

        # DBキャッシュ
        if self.use_db_cache:
            cur = self.conn.cursor()
            cur.execute(
                "SELECT length FROM paths WHERE coord1=? AND coord2=?",
                (str(node1), str(node2))
            )
            hit = cur.fetchone()
            if hit is not None:
                length = hit[0]
                if node1 not in self.len_dic:
                    type(self).len_dic[node1] = {}
                if length is not None:
                    type(self).len_dic[node1][node2] = length
                    # 逆方向も保存
                    if node2 not in self.len_dic:
                        type(self).len_dic[node2] = {}
                    type(self).len_dic[node2][node1] = length
                    return length
                else:
                    return None

        # 計算
        dist = self._dijkstra(node1, node2)
        if node1 not in self.len_dic:
            type(self).len_dic[node1] = {}
        if node2 not in self.len_dic:
            type(self).len_dic[node2] = {}
        type(self).len_dic[node1][node2] = dist
        type(self).len_dic[node2][node1] = dist  # ← 逆方向も格納

        # DBへのバッファ
        if self.use_db_cache:
            self._bulk_buffer.append((str(node1), str(node2), dist))
            self._bulk_buffer.append((str(node2), str(node1), dist))  # DBも逆方向追記
            if len(self._bulk_buffer) >= self.bulk_size:
                self._flush_bulk()
        
        return dist

    def _dijkstra(self, start, goal):
        try:
            return nx.dijkstra_path_length(self.G, source=start, target=goal)
        except nx.NetworkXNoPath:
            print("SP error")
            return None

    def _flush_bulk(self):
        if self.use_db_cache and self._bulk_buffer:
            cur = self.conn.cursor()
            cur.executemany(
                "INSERT OR IGNORE INTO paths (coord1, coord2, length) VALUES (?, ?, ?)",
                self._bulk_buffer
            )
            self.conn.commit()
            self._bulk_buffer = []

    def close(self):
        self._flush_bulk()
        if self.conn:
            self.conn.close()

class Routing:

    def __init__(self, sp):
        self.sp = sp
        self.G = self.sp.G 
        self.len_dic = self.sp.len_dic 
        self.counter = itertools.count()  # プライオリティキュー用カウンタ

    def SPC(self, s, t):
        return self.sp.len_SP(s, t) 

    def init_vehicle(self, Q, st, SPC_cache):
        d = {}
        pie = {}
        T = {}
        mark = {}
        stop = {}

        for node in self.G.nodes:
            d[node] = float('inf')
            pie[node] = None
            T[node] = set()
            mark[node] = False
            stop[node] = False

        d[st] = 0
        T[st].add((st, 0)) 

        for q in Q:
            cost_s_st = SPC_cache.get((q[0], st), float('inf'))
            cost_d_st = SPC_cache.get((q[1], st), float('inf')) #ここが原因 最初でこれらのコストがinfだと成立しない？
            d[st] += cost_s_st + cost_d_st
            T[st].add((q[0], cost_s_st))
            T[st].add((q[1], cost_d_st))

        return d, pie, T, mark, stop

    def relax_vehicle(self, u, v, c, st, Q, SPC_cache, d, pie, T, PQ):
        print(f"\n--- relax_vehicle: u={u}, v={v}, edge_cost={c}")
        if u not in T or not any(item[0] == st for item in T[u]):
            print("[ERROR] u not in T or st未登録")
            return d, pie, T, PQ

        vehicle_cost_pair = next(item for item in T[u] if item[0] == st)
        cv = vehicle_cost_pair[1]
        d_temp_v = cv + c
        T_temp_v = set()
        T_temp_v.add((st, cv + c))

        print(f"  車両コスト cv={cv}, d_temp_v初期={d_temp_v}")

        for si, di in Q:
            cs, cd = None, None
            if u in T:
                for tname, tcost in T[u]:
                    if tname == si:
                        cs = tcost
                    elif tname == di:
                        cd = tcost

            weight = 1.0
            cost_si_v = SPC_cache.get((si, v), float('inf')) * weight
            cost_di_v = SPC_cache.get((di, v), float('inf')) * weight

            print(f"  - si={si}, di={di}, cs={cs}, cd={cd}, cost_si_v={cost_si_v}, cost_di_v={cost_di_v}")

            if cost_si_v == float('inf') or cost_di_v == float('inf'):
                print(f"    脱落（inf）: siかdiがinf")
                return d, pie, T, PQ

            min_cd_cdi = min(cd if cd is not None else float('inf'), cost_di_v)

            print(f"    min_cd_cdi={min_cd_cdi}")

            if cs is not None and cd is not None and (cs + min_cd_cdi) < (cost_si_v + cost_di_v):
                if cs == float('inf') or min_cd_cdi == float('inf'):
                    print(f"    脱落（内部値がinf）")
                    return d, pie, T, PQ
                d_temp_v += cs + min_cd_cdi
                T_temp_v.add((si, cs))
                T_temp_v.add((di, min_cd_cdi))
                print(f"    ->既乗車ベース cs+min_cd_cdi採用. d_temp_v={d_temp_v}")
            else:
                d_temp_v += cost_si_v + cost_di_v
                T_temp_v.add((si, cost_si_v))
                T_temp_v.add((di, cost_di_v))
                print(f"    ->新規経路 cost_si_v+cost_di_v採用. d_temp_v={d_temp_v}")

        print(f"  比較: d_temp_v={d_temp_v} < d[{v}]={d[v]}")
        if d_temp_v < d[v]:
            d[v] = d_temp_v
            pie[v] = u
            T[v] = T_temp_v
            print(f"  PQ追加: cost={d[v]}, node={v}, T[v]={T[v]}")
            heapq.heappush(PQ, (d[v], next(self.counter), v))
        else:
            print(f"  更新せず(d[{v}]), {d[v]}≦{d_temp_v}")

        print(f"  T[{v}] = {T[v] if v in T else None}")
        print(f"  PQ状態: {[ (cost, node) for (cost, _, node) in sorted(PQ)] }")
        return d, pie, T, PQ

    def vehicle_stops(self, st, en, pie, Q, SPC_cache):
        # st, en, pie, Qの中すべてstr
        if st not in self.G.nodes or en not in self.G.nodes:
            return []
        path_list = []
        current = en
        while current is not None:
            path_list.append(current)
            if current == st:
                break
            current = pie.get(current)
        path_list.reverse()
        if not path_list or path_list[0] != st or path_list[-1] != en:
            return []

        path_node_to_index = {node: idx for idx, node in enumerate(path_list)}

        stop_points = {node: False for node in path_list}
        for si, di in Q:
            if si in self.G.nodes and di in self.G.nodes:
                min_dist_si = float('inf'); nearest_si = None
                min_dist_di = float('inf'); nearest_di = None
                for vp_node in path_list:
                    d_si = SPC_cache.get((si, vp_node), float('inf'))
                    if d_si < min_dist_si:
                        min_dist_si = d_si
                        nearest_si = vp_node
                    elif d_si == min_dist_si:
                        if path_node_to_index[vp_node] < path_node_to_index.get(nearest_si, 1e9):
                            nearest_si = vp_node
                    d_di = SPC_cache.get((di, vp_node), float('inf'))
                    if d_di < min_dist_di:
                        min_dist_di = d_di
                        nearest_di = vp_node
                    elif d_di == min_dist_di:
                        if path_node_to_index[vp_node] < path_node_to_index.get(nearest_di, 1e9):
                            nearest_di = vp_node
                if nearest_si is not None:
                    stop_points[nearest_si] = True
                if nearest_di is not None:
                    stop_points[nearest_di] = True

        stops_path = []
        for node in path_list:
            if node == st or node == en or stop_points[node]:
                stops_path.append(node)
        return stops_path

    def find_optimal_stops(self, Q, st, en, R1=float('inf')):
        SPC_cache = {}

        query_endpoints = set(n for q in Q for n in q)
        endpoints = query_endpoints | {st, en}
        all_nodes = list(self.G.nodes)

        #徒歩移動距離制約
        LN = float('inf')

        for source in endpoints:
            for target in all_nodes:
                dist = self.sp.len_SP(source, target)
                SPC_cache[(source, target)] = dist

        d, pie, T, mark, _ = self.init_vehicle(Q, st, SPC_cache)
        
        PQ = []
        for node in self.G.nodes:
            if d[node] != float('inf'):
                heapq.heappush(PQ, (d[node], next(self.counter), node))

        print("[初期PQ全体]", [(cost, node) for cost, _, node in sorted(PQ)])

        step = 0
        while PQ:
            print("\n" + "="*30)
            print(f"[step={step}] PQ全体: {[ (cost, node) for (cost, _, node) in sorted(PQ)] }")
            current_distance, cnt, u = heapq.heappop(PQ)
            print(f">>> PQpop: cost={current_distance}, counter={cnt}, node={u}")
            print(f"  現在d[u] = {d[u]}")
            if current_distance > d[u]:
                print(f"  (スキップ理由: PQコスト{current_distance} > 現状d[{u}]={d[u]})")
                continue
            mark[u] = True

            for v in list(self.G.neighbors(u)):
                if not mark[v]:
                    weight = self.G[u][v].get('weight', 1)
                    print(f"  --- neighbor v={v}: G[{u}][{v}]['weight']={weight}")
                    d, pie, T, PQ = self.relax_vehicle(u, v, weight, st, Q, SPC_cache, d, pie, T, PQ)
            step += 1

        print("\n=== [ノードごとの車両コストとユーザソロコスト状態一覧] ===")
        for node in self.G.nodes:
            vehicle_cost = None
            if node in T:
                for tname, tcost in T[node]:
                    if tname == st:
                        vehicle_cost = int(tcost)
                        break
            cost_pairs = []
            if node in T:
                for si, di in Q:
                    solo_cost_in = solo_cost_out = None
                    for tname, tcost in T[node]:
                        if tname == si:
                            solo_cost_in = int(tcost)
                        elif tname == di:
                            solo_cost_out = int(tcost)
                    cost_pairs.append(f"({solo_cost_in if solo_cost_in is not None else 0},{solo_cost_out if solo_cost_out is not None else 0})")
            total_cost = int(d[node]) if d[node] != float('inf') else 'inf'
            vehicle_cost = vehicle_cost if vehicle_cost is not None else 0
            print(f"{node}: {vehicle_cost}|{{{','.join(cost_pairs)}}}|{total_cost}")

        print("\n=== [最終d] ===")
        for k in d: print(f"  {k}: {d[k]}")
        print("=== [最終pie] ===")
        for k in pie: print(f"  {k}: {pie[k]}")

        P = self.vehicle_stops(st, en, pie, Q, SPC_cache)
        print("=== [車両path（停車箇所）] ===")
        print(P)
        return P
    
import networkx as nx

G = nx.Graph()
G.add_edge("a", "b", weight=1)
G.add_edge("a", "c", weight=3)
G.add_edge("b", "c", weight=5)
G.add_edge("b", "d", weight=2)
G.add_edge("c", "d", weight=5)
G.add_edge("c", "e", weight=7)
G.add_edge("d", "e", weight=6)
G.add_edge("d", "f", weight=4)
G.add_edge("e", "f", weight=1)
G.add_edge("e", "g", weight=8)
G.add_edge("f", "g", weight=3)
G.add_edge("f", "h", weight=4)
G.add_edge("g", "h", weight=2)

Q = [("b", "f"), ("c", "g")]

sp = ShortestPathFinder(G)
R = Routing(sp)
P = R.find_optimal_stops(Q, "a", "h")

graph constructed
[初期PQ全体] [(21, 'a')]

[step=0] PQ全体: [(21, 'a')]
>>> PQpop: cost=21, counter=0, node=a
  現在d[u] = 21
  --- neighbor v=b: G[a][b]['weight']=1

--- relax_vehicle: u=a, v=b, edge_cost=1
  車両コスト cv=0, d_temp_v初期=1
  - si=b, di=f, cs=1, cd=7, cost_si_v=0.0, cost_di_v=6.0
    min_cd_cdi=6.0
    ->新規経路 cost_si_v+cost_di_v採用. d_temp_v=7.0
  - si=c, di=g, cs=3, cd=10, cost_si_v=4.0, cost_di_v=9.0
    min_cd_cdi=9.0
    ->既乗車ベース cs+min_cd_cdi採用. d_temp_v=19.0
  比較: d_temp_v=19.0 < d[b]=inf
  PQ追加: cost=19.0, node=b, T[v]={('a', 1), ('f', 6.0), ('c', 3), ('g', 9.0), ('b', 0.0)}
  T[b] = {('a', 1), ('f', 6.0), ('c', 3), ('g', 9.0), ('b', 0.0)}
  PQ状態: [(19.0, 'b')]
  --- neighbor v=c: G[a][c]['weight']=3

--- relax_vehicle: u=a, v=c, edge_cost=3
  車両コスト cv=0, d_temp_v初期=3
  - si=b, di=f, cs=1, cd=7, cost_si_v=4.0, cost_di_v=8.0
    min_cd_cdi=7
    ->既乗車ベース cs+min_cd_cdi採用. d_temp_v=11
  - si=c, di=g, cs=3, cd=10, cost_si_v=0.0, cost_di_v=11.0
    min_cd_cdi=10
    ->新規経路 cost_si

In [47]:
import heapq
import itertools
import sqlite3
import networkx as nx
import ast

class ShortestPathFinder:
    len_dic = {}
    kdtree = None
    kdtree_nodes = None
    kdtree_coords_km = None
    transformer = None
    zone = None

    def __init__(self, G, use_db_cache=False, db_path="paths.db", weight="weight", bulk_size=100, utm_zone=54):
        self.G = G
        self.weight = weight
        self.use_db_cache = use_db_cache
        self.bulk_size = bulk_size
        self._bulk_buffer = []
        self.zone = utm_zone

        if self.use_db_cache:
            self.conn = sqlite3.connect(db_path)
            self._init_db()
        else:
            self.conn = None
        print("graph constructed")

    def _init_db(self):
        cur = self.conn.cursor()
        cur.execute('''
            CREATE TABLE IF NOT EXISTS paths (
                coord1 TEXT,
                coord2 TEXT,
                length REAL,
                PRIMARY KEY (coord1, coord2)
            )
        ''')
        cur.execute('CREATE INDEX IF NOT EXISTS idx_coord1 ON paths(coord1)')
        cur.execute('CREATE INDEX IF NOT EXISTS idx_pair ON paths(coord1, coord2)')
        self.conn.commit()

    def _latlon_to_km(self, latlon):
        x, y = type(self).transformer.transform(latlon[1], latlon[0])
        return [x/1000, y/1000]

    def nearestNode(self, p):
        coord_km = self._latlon_to_km(p)
        dist, idx = type(self).kdtree.query(coord_km)
        return type(self).kdtree_nodes[idx]

    def nodes_within_radius(self, p, r_km):
        latlon = ast.literal_eval(p)
        coord_km = self._latlon_to_km(latlon)
        idxs = type(self).kdtree.query_ball_point(coord_km, r_km)
        return [type(self).kdtree_nodes[i] for i in idxs]

    def euclidean(self, a, b):
        a = list(map(float, a.strip("[]").split(',')))
        b = list(map(float, b.strip("[]").split(',')))
        return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

    def SP(self, st, en):
        return nx.astar_path(self.G, st, en, heuristic=self.euclidean)

    def len_SP(self, node1, node2):
        try:
            return type(self).len_dic[node1][node2]
        except KeyError:
            pass

        if self.use_db_cache:
            cur = self.conn.cursor()
            cur.execute(
                "SELECT length FROM paths WHERE coord1=? AND coord2=?",
                (str(node1), str(node2))
            )
            hit = cur.fetchone()
            if hit is not None:
                length = hit[0]
                if node1 not in self.len_dic:
                    type(self).len_dic[node1] = {}
                if length is not None:
                    type(self).len_dic[node1][node2] = length
                    if node2 not in self.len_dic:
                        type(self).len_dic[node2] = {}
                    type(self).len_dic[node2][node1] = length
                    return length
                else:
                    return None

        dist = self._dijkstra(node1, node2)
        if node1 not in self.len_dic:
            type(self).len_dic[node1] = {}
        if node2 not in self.len_dic:
            type(self).len_dic[node2] = {}
        type(self).len_dic[node1][node2] = dist
        type(self).len_dic[node2][node1] = dist

        if self.use_db_cache:
            self._bulk_buffer.append((str(node1), str(node2), dist))
            self._bulk_buffer.append((str(node2), str(node1), dist))
            if len(self._bulk_buffer) >= self.bulk_size:
                self._flush_bulk()

        return dist

    def _dijkstra(self, start, goal):
        try:
            return nx.dijkstra_path_length(self.G, source=start, target=goal)
        except nx.NetworkXNoPath:
            print("SP error")
            return None

    def _flush_bulk(self):
        if self.use_db_cache and self._bulk_buffer:
            cur = self.conn.cursor()
            cur.executemany(
                "INSERT OR IGNORE INTO paths (coord1, coord2, length) VALUES (?, ?, ?)",
                self._bulk_buffer
            )
            self.conn.commit()
            self._bulk_buffer = []

    def close(self):
        self._flush_bulk()
        if self.conn:
            self.conn.close()


class Routing:

    def __init__(self, sp):
        self.sp = sp
        self.G = self.sp.G
        self.len_dic = self.sp.len_dic
        self.counter = itertools.count()

    def SPC(self, s, t):
        return self.sp.len_SP(s, t)

    def init_vehicle(self, Q, st, SPC_cache, R1):
        d = {}
        pie = {}
        T = {}
        mark = {}
        stop = {}
        for node in self.G.nodes:
            d[node] = 0 if node == st else float('inf')
            pie[node] = None
            T[node] = set()
            mark[node] = False
            stop[node] = False
        T[st].add((st, 0))
        for q in Q:
            cost_s_st = SPC_cache.get((q[0], st), float('inf'))
            cost_d_st = SPC_cache.get((q[1], st), float('inf'))
            if cost_s_st > R1 or cost_d_st > R1:
                T[st].add((q[0], float('inf')))
                T[st].add((q[1], float('inf')))
                continue  # ここでd[st]に何も加えず
            d[st] += cost_s_st + cost_d_st
            T[st].add((q[0], cost_s_st))
            T[st].add((q[1], cost_d_st))
        return d, pie, T, mark, stop

    def relax_vehicle(self, u, v, c, st, Q, SPC_cache, d, pie, T, PQ, R1):
        if u not in T or not any(item[0] == st for item in T[u]):
            return d, pie, T, PQ

        vehicle_cost_pair = next(item for item in T[u] if item[0] == st)
        cv = vehicle_cost_pair[1]
        d_temp_v = cv + c
        T_temp_v = set()
        T_temp_v.add((st, cv + c))

        for si, di in Q:
            cs, cd = None, None
            if u in T:
                for tname, tcost in T[u]:
                    if tname == si:
                        cs = tcost
                    elif tname == di:
                        cd = tcost
            cost_si_v = SPC_cache.get((si, v), float('inf'))
            cost_di_v = SPC_cache.get((di, v), float('inf'))
            # R1超なら今回はこのユーザは乗降不可
            if cost_si_v > R1 or cost_di_v > R1:
                T_temp_v.add((si, float('inf')))
                T_temp_v.add((di, float('inf')))
                continue

            min_cd_cdi = min(cd if cd is not None else float('inf'), cost_di_v)

            if cs is not None and cd is not None and (cs + min_cd_cdi) < (cost_si_v + cost_di_v):
                if cs == float('inf') or min_cd_cdi == float('inf'):
                    return d, pie, T, PQ
                d_temp_v += cs + min_cd_cdi
                T_temp_v.add((si, cs))
                T_temp_v.add((di, min_cd_cdi))
            else:
                d_temp_v += cost_si_v + cost_di_v
                T_temp_v.add((si, cost_si_v))
                T_temp_v.add((di, cost_di_v))

        if d_temp_v < d[v]:
            d[v] = d_temp_v
            pie[v] = u
            T[v] = T_temp_v
            heapq.heappush(PQ, (d[v], next(self.counter), v))

        return d, pie, T, PQ

    def vehicle_stops(self, st, en, pie, Q, SPC_cache, R1):
        def insert_node_into_path(G, path, node):
            min_extra = float('inf')
            best_path = None
            for i in range(len(path)):
                new_path = path[:i+1] + [node] + path[i+1:]
                try:
                    total = 0
                    for k in range(len(new_path)-1):
                        total += nx.dijkstra_path_length(G, new_path[k], new_path[k+1])
                    if total < min_extra:
                        min_extra = total
                        best_path = new_path
                except nx.NetworkXNoPath:
                    continue
            return best_path if best_path is not None else path

        if st not in self.G.nodes or en not in self.G.nodes:
            return []

        # 経路path復元
        path_list = []
        current = en
        while current is not None:
            path_list.append(current)
            if current == st:
                break
            current = pie.get(current)
        path_list.reverse()
        if not path_list or path_list[0] != st or path_list[-1] != en:
            return []

        # --- 必要なノード（到達不可能ノード）を一度に全部追加 ---
        while True:
            nodes_to_insert = []
            for si, di in Q:
                # si
                found_si = any(SPC_cache.get((si, v), float('inf')) <= R1 for v in path_list)
                if not found_si and si not in path_list:
                    nodes_to_insert.append(si)
                # di
                found_di = any(SPC_cache.get((di, v), float('inf')) <= R1 for v in path_list)
                if not found_di and di not in path_list:
                    nodes_to_insert.append(di)
            if not nodes_to_insert:
                break
            for node in nodes_to_insert:
                path_list = insert_node_into_path(self.G, path_list, node)
        # -------------------------------------------------------

        # 停車点選定はこれまで通り
        path_node_to_index = {node: idx for idx, node in enumerate(path_list)}
        stop_points = {node: False for node in path_list}
        for si, di in Q:
            min_dist_si = float('inf'); nearest_si = None
            min_dist_di = float('inf'); nearest_di = None
            for vp_node in path_list:
                d_si = SPC_cache.get((si, vp_node), float('inf'))
                if d_si > R1: continue
                if d_si < min_dist_si:
                    min_dist_si = d_si
                    nearest_si = vp_node
                elif d_si == min_dist_si:
                    if path_node_to_index[vp_node] < path_node_to_index.get(nearest_si, 1e9):
                        nearest_si = vp_node
                d_di = SPC_cache.get((di, vp_node), float('inf'))
                if d_di > R1: continue
                if d_di < min_dist_di:
                    min_dist_di = d_di
                    nearest_di = vp_node
                elif d_di == min_dist_di:
                    if path_node_to_index[vp_node] < path_node_to_index.get(nearest_di, 1e9):
                        nearest_di = vp_node
            if nearest_si is not None: stop_points[nearest_si] = True
            if nearest_di is not None: stop_points[nearest_di] = True

        stops_path = []
        for node in path_list:
            if node == st or node == en or stop_points[node]:
                stops_path.append(node)
        return stops_path

    def find_optimal_stops(self, Q, st, en, R1=float('inf')):
        SPC_cache = {}
        query_endpoints = set(n for q in Q for n in q)
        endpoints = query_endpoints | {st, en}
        all_nodes = list(self.G.nodes)

        for source in endpoints:
            for target in all_nodes:
                dist = self.sp.len_SP(source, target)
                SPC_cache[(source, target)] = dist

        d, pie, T, mark, _ = self.init_vehicle(Q, st, SPC_cache, R1)

        PQ = []
        for node in self.G.nodes:
            if d[node] != float('inf'):
                heapq.heappush(PQ, (d[node], next(self.counter), node))

        while PQ:
            current_distance, cnt, u = heapq.heappop(PQ)
            if current_distance > d[u]:
                continue
            mark[u] = True
            for v in list(self.G.neighbors(u)):
                if not mark[v]:
                    weight = self.G[u][v].get('weight', 1)
                    d, pie, T, PQ = self.relax_vehicle(u, v, weight, st, Q, SPC_cache, d, pie, T, PQ, R1)

        print("\n=== [最終d] ===")
        for k in d: print(f"  {k}: {d[k]}")
        print("=== [最終pie] ===")
        for k in pie: print(f"  {k}: {pie[k]}")

        P = self.vehicle_stops(st, en, pie, Q, SPC_cache, R1)
        print("=== [車両path（停車箇所）] ===")
        print(P)
        return P

# テスト例
G = nx.Graph()
G.add_edge("a", "b", weight=1)
G.add_edge("a", "c", weight=3)
G.add_edge("b", "c", weight=5)
G.add_edge("b", "d", weight=2)
G.add_edge("c", "d", weight=5)
G.add_edge("c", "e", weight=7)
G.add_edge("d", "e", weight=6)
G.add_edge("d", "f", weight=4)
G.add_edge("e", "f", weight=1)
G.add_edge("e", "g", weight=8)
G.add_edge("f", "g", weight=3)
G.add_edge("f", "h", weight=4)
G.add_edge("g", "h", weight=2)

Q = [("b", "f"), ("c", "g")]

sp = ShortestPathFinder(G)
R = Routing(sp)
P = R.find_optimal_stops(Q, "a", "h", R1=1)

graph constructed

=== [最終d] ===
  a: 0
  b: 1
  c: 3
  d: 3
  e: 8
  f: 7
  g: 10
  h: 11
=== [最終pie] ===
  a: None
  b: a
  c: a
  d: b
  e: f
  f: d
  g: f
  h: f
=== [車両path（停車箇所）] ===
['a', 'c', 'b', 'h']
